In [1]:
import pandas as pd
import torch
from datasets import Dataset
import datasets
import os
import random
import numpy as np

def seedeverything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True  ##
    torch.backends.cudnn.benchmark = True

seedeverything(seed=233)

traindata = pd.read_json('./CHIP-CTC/CHIP-CTC_train.json')
valdata = pd.read_json('./CHIP-CTC/CHIP-CTC_dev.json')
testdata = pd.read_json('./CHIP-CTC/CHIP-CTC_test.json')

examplepreddata = pd.read_excel('./CHIP-CTC/category.xlsx')

examplepreddata['label2idx'] = range(examplepreddata.shape[0])

label2idx = dict(
    zip(examplepreddata['Label Name'], examplepreddata['label2idx']))
idx2label = dict(
    zip(examplepreddata['label2idx'], examplepreddata['Label Name']))

print(idx2label)

traindata['labels'] = [label2idx[item] for item in traindata['label']]
valdata['labels'] = [label2idx[item] for item in valdata['label']]

print(len(traindata))
print(len(valdata))
print(len(testdata))

traindataset = Dataset.from_pandas(traindata)
valdataset = Dataset.from_pandas(valdata)
testdataset = Dataset.from_pandas(testdata)

dataset = datasets.DatasetDict({
    'train': traindataset,
    'validation': valdataset,
    'test': testdataset
})

print(dataset)

train_dataset = dataset['train']
print(train_dataset.features)

print(train_dataset[0])

/home/gray/.conda/envs/python37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{0: 'Disease', 1: 'Symptom', 2: 'Sign', 3: 'Pregnancy-related Activity', 4: 'Neoplasm Status', 5: 'Non-Neoplasm Disease Stage', 6: 'Allergy Intolerance', 7: 'Organ or Tissue Status', 8: 'Life Expectancy', 9: 'Oral related', 10: 'Pharmaceutical Substance or Drug', 11: 'Therapy or Surgery', 12: 'Device', 13: 'Nursing', 14: 'Diagnostic', 15: 'Laboratory Examinations', 16: 'Risk Assessment', 17: 'Receptor Status', 18: 'Age', 19: 'Special Patient Characteristic', 20: 'Literacy', 21: 'Gender', 22: 'Education', 23: 'Address', 24: 'Ethnicity', 25: 'Consent', 26: 'Enrollment in other studies', 27: 'Researcher Decision', 28: 'Capacity', 29: 'Ethical Audit', 30: 'Compliance with Protocol', 31: 'Addictive Behavior', 32: 'Bedtime', 33: 'Exercise', 34: 'Diet', 35: 'Alcohol Consumer', 36: 'Sexual related', 37: 'Smoking Status', 38: 'Blood Donation', 39: 'Encounter', 40: 'Disabilities', 41: 'Healthy', 42: 'Data Accessible', 43: 'Multiple'}
22962
7682
10192
DatasetDict({
    train: Dataset({
        fe

In [2]:
from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('uer/chinese_roberta_L-8_H-512',
                                          mirror='tuna')

def tokenize_function(sample):
    return tokenizer(sample['text'], truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets['train'] = tokenized_datasets['train'].remove_columns(
    ['id', 'text', 'label'])
tokenized_datasets['validation'] = tokenized_datasets[
    'validation'].remove_columns(['id', 'text', 'label'])
tokenized_datasets['test'] = tokenized_datasets['test'].remove_columns(
    ['id', 'text'])

from transformers import DataCollatorWithPadding  #实现按batch自动padding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print(tokenized_datasets)

Parameter 'function'=<function tokenize_function at 0x7f1e580e07a0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 11/11 [00:01<00:00,  8.50ba/s]


DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 22962
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7682
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10192
    })
})


In [3]:
from torch.utils.data import DataLoader, Dataset
train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=32, collate_fn=data_collator)  
val_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=32, collate_fn=data_collator)
test_dataloader = DataLoader(tokenized_datasets['test'], batch_size=32, collate_fn=data_collator)
for batch in test_dataloader:
    print({k: v.shape for k, v in batch.items()})
    break

{'input_ids': torch.Size([32, 76]), 'token_type_ids': torch.Size([32, 76]), 'attention_mask': torch.Size([32, 76])}


In [4]:
from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification

checkpoint = 'uer/chinese_roberta_L-8_H-512'
net = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=examplepreddata.shape[0])

for batch in train_dataloader:
    outputs = net(**batch)
    print(outputs)
    break

Some weights of the model checkpoint at uer/chinese_roberta_L-8_H-512 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at uer/chinese_rob

SequenceClassifierOutput(loss=tensor(3.7834, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.0997,  0.0356,  0.0691,  ..., -0.0297,  0.0373,  0.1272],
        [ 0.0572,  0.0843,  0.0407,  ..., -0.0085,  0.0295,  0.1387],
        [ 0.1198,  0.0831,  0.1267,  ..., -0.0696,  0.0629,  0.0540],
        ...,
        [ 0.0499, -0.0527,  0.0166,  ...,  0.0521,  0.0730,  0.0817],
        [ 0.0578,  0.0536,  0.1319,  ..., -0.0502,  0.0895,  0.0580],
        [ 0.0762, -0.0394,  0.0129,  ...,  0.0482,  0.0537,  0.0232]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [12]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import AdamW, get_scheduler
from datasets import load_metric
from statistics import mean
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 8
lr = 3e-5
num_training_steps = num_epochs * len(train_dataloader)  # num of batches * num of epochs
print(num_training_steps)


class Bert4textclassification_lightningsystem(pl.LightningModule):

    def __init__(self,net,lr,epoch,len):
        super(Bert4textclassification_lightningsystem, self).__init__()
        self.net = net.to(device)
        self.lr = lr
        self.epoch = epoch
        self.num_training_steps = len
        #self.metric = load_metric("glue", "mrpc",mirror="tuna")
        
    def configure_optimizers(self):

        self.optimizer = AdamW(self.net.parameters(), lr=self.lr)
        lr_scheduler = get_scheduler(
                'linear',
                optimizer=self.optimizer, 
                num_warmup_steps=0,
                num_training_steps=self.num_training_steps)
        optim_dict = {'optimizer': self.optimizer, 'lr_scheduler': lr_scheduler}
        return optim_dict
    def metrics_compute(self,mode,outputs):
        loss = []
        loss.append(outputs[0][mode+'_loss'])
        predictions = outputs[0]['predictions']
        labels = outputs[0]['labels']
        for i in range(1,len(outputs)):
            loss.append(outputs[i][mode+'_loss'])
            predictions = torch.concat([predictions,outputs[i]['predictions']],dim=0)
            labels = torch.concat([labels,outputs[i]['labels']],dim=0)
        loss = torch.tensor(loss)
        predictions = predictions.cpu().detach().numpy()
        labels = labels.cpu().detach().numpy()
        return loss,predictions,labels

    def training_step(self, batch, batch_idx):
        batch = {k:v.to(device) for k,v in batch.items()}
        loss = self.net(**batch).loss
        return loss

    def validation_step(self, batch, batch_idx):
        batch = {k:v.to(device) for k,v in batch.items()}
        outputs = self.net(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metrics_dict = metrics.classification_report(predictions.cpu().detach().numpy(),batch['labels'].cpu().detach().numpy(),digits = 4,output_dict=True)
        self.log('val_weighted_f1',metrics_dict['weighted avg']['f1-score'])
        #self.metric.add_batch(predictions=predictions, references=batch["labels"])
        return {'val_loss':outputs.loss,'predictions':predictions,'labels':batch['labels']}
    
       
    def test_step(self, batch, batch_idx):
        batch = {k:v.to(device) for k,v in batch.items()}
        outputs = self.net(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        return {'test_loss':outputs.loss,'predictions':predictions}
    
    def training_epoch_end(self,outputs):
        pass

    def validation_epoch_end(self, outputs):
        print(outputs[0]['predictions'].shape)
        print(len(outputs))
        val_loss ,predictions,labels= self.metrics_compute('val',outputs)
        print(predictions.shape)
        print('\n',"val_loss: ",val_loss.mean())
        print(metrics.classification_report(predictions, labels,digits = 4))

    def test_epoch_end(self, outputs):
        predictions = outputs[0]['predictions']
        for i in range(1,len(outputs)):
            predictions = torch.concat([predictions,outputs[i]['predictions']],dim=0)
        predictions = predictions.cpu().detach().numpy().tolist()
        test_labels =[ idx2label[idx] for idx in predictions]
        testdata['label'] = test_labels
        test_pred_list = []
        for i in range(testdata.shape[0]):
            temp_dict = {}
            temp_dict['id'] = testdata.iloc[i,0]
            temp_dict['label'] = testdata.iloc[i,2]
            temp_dict['text'] = testdata.iloc[i,1]
            test_pred_list.append(temp_dict)
        print('\n',testdata.head())
        print(temp_dict)
        testdata.to_json('results_1.json')


5744


In [6]:
model = Bert4textclassification_lightningsystem(net,lr,num_epochs,num_training_steps)
checkpoint_callback = ModelCheckpoint(
            monitor='val_weighted_f1',
            dirpath='./output',
            filename=
            'chinese_roberta_L-8_H-512-CHIP-CTC-{epoch:02d}-{val_weighted_f1:.3f}',
            mode='max')
trainer = Trainer(
            logger=False,
            max_epochs=num_epochs,
            gpus=1,
            reload_dataloaders_every_n_epochs=False,
            num_sanity_val_steps=0,  # Skip Sanity Check
            callbacks=[checkpoint_callback],
            #limit_train_batches=0.05
            #precision=16,
            #accumulate_grad_batches=2,
            #gradient_clip_val=0.5,
        )

trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type                          | Params
-------------------------------------------------------
0 | net  | BertForSequenceClassification | 36.6 M
-------------------------------------------------------
36.6 M    Trainable params
0         Non-trainable params
36.6 M    Total params
146.344   Total estimated model params size (MB)


Epoch 0:   1%|          | 10/959 [00:00<01:08, 13.83it/s, loss=3.46]

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.

In [13]:

model = Bert4textclassification_lightningsystem.load_from_checkpoint(checkpoint_path='./output/chinese_roberta_L-8_H-512-CHIP-CTC-epoch=02-val_weighted_f1=0.843.ckpt',
net=net,lr=lr,epoch=num_epochs,len=num_training_steps
        )

trainer = Trainer(
            logger=False,
            gpus=1,
            #limit_train_batches=0.05
            #precision=16,
            #accumulate_grad_batches=2,
            #gradient_clip_val=0.5,
        )
trainer.test(model=model, dataloaders=test_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 319/319 [00:04<00:00, 73.05it/s]
    id                                     text               label
0  s1                         4.手术体位不适合BIS 监测；              Device
1  s2                            5.术前未能开放静脉通路。  Therapy or Surgery
2  s3  1.年龄18岁至40岁有生育要求的重度宫腔粘连（IV度、Va度及Vb度）患者；            Multiple
3  s4             2.生殖系统无器质性病变，术前病理检查无子宫内膜病变者；             Disease
4  s5                          3.同意并签署患者知情同意书；             Consent
Testing DataLoader 0: 100%|██████████| 319/319 [00:04<00:00, 72.72it/s]


[{}]